# Categorical Boost
#### cv score :  0.55482797731569 (5 folds)

In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import sklearn
from Functions import prepro
import warnings

import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from lightgbm import LGBMClassifier


random.seed(0)

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv("../../data/Processed/train2.csv", index_col='id')
test_df = pd.read_csv("../../data/Processed/test2.csv", index_col="id")
print(train_df.shape)
print(test_df.shape)

(132250, 192)
(28340, 191)


### 武器の情報を個人ごとからチームごとに変える
category1, 2を落とす、

他のカテゴリをチームごとにする

reskinそのまま

In [ ]:
# 落とさないほうが精度上がってしまった。。。

"""suffixes = ["-A1", "-A2", "-A3", "-A4", "-B1", "-B2", "-B3", "-B4"]
drop_cols = ["category1", "category2"]

for c in drop_cols:
    for s in suffixes:
        col = c+s
        train_df.drop(col, axis=1, inplace=True)
        test_df.drop(col, axis=1, inplace=True)
"""

In [3]:
y = train_df["y"]
train_df = train_df.drop("y", axis=1)

In [4]:
# add info about numeric column

num_cols = ["level", "range", "rapid-", "atack"]

#train_df = prepro.flat(train_df, num_cols)
#test_df = prepro.flat(test_df, num_cols)

train_df = prepro.add_numeric_info(train_df, num_cols)
test_df = prepro.add_numeric_info(test_df, num_cols)

In [5]:
# add team info

train_df,  test_df = prepro.addTeamInfo(train_df, test_df, cols=["special", "subweapon", "category1", "category2", "mainweapon"])

(160590, 239)
special-A
special-B
subweapon-A
subweapon-B
category1-A
category1-B
category2-A
category2-B
mainweapon-A
mainweapon-B
(160590, 423)
complete


In [6]:
# add disconnection info

train_df = prepro.add_disconnection(train_df)
test_df = prepro.add_disconnection(test_df)

In [7]:
# categorize team

categorize_col = ["category1", "category2", "subweapon", "special", "mainweapon"]
for col in categorize_col:
    print(col)
    train_df_, test_df_ = prepro.categorize_team(train_df, test_df, col)

category1
category2
subweapon
special
mainweapon


In [8]:
#target encoding

#cat_tgtenc_cols = ["mode", "stage", "team-category1-A", "team-category1-B", 
#                  "team-category2-A", "team-category2-B", "team-subweapon-A", "team-subweapon-B", "team-special-A", "team-special-B"]

cat_tgtenc_cols = ["mode", "stage", "team-category1-A", "team-category1-B"]
for col in cat_tgtenc_cols:
    print(col)
    train_df, test_df = prepro.target_encoding(train_df, test_df, y, col)
    

mode
fold 0


../Functions/prepro.py:187: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  indices = df1_.iloc[fold][df1_[col] == content].index-1  # fold内のある種別を持つインデックスの抽出
../Functions/prepro.py:188: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tgt_sum = df1_.iloc[out_fold][df1_[col] == content]["y"].sum()
../Functions/prepro.py:189: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tgt_size = df1_.iloc[out_fold][df1_[col] == content]["y"].shape[0]
/opt/anaconda3/envs/spla/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


fold 1
fold 2
fold 3
fold 4


../Functions/prepro.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[tgt_col][df2[col] == content] = df1[df1[col] == content][tgt_col].mean()
../Functions/prepro.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[tgt_col][df2[col] == content] = df1[df1[col] == content][tgt_col].mean()


stage
fold 0


...

../Functions/prepro.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[tgt_col][df2[col] == content] = df1[df1[col] == content][tgt_col].mean()


team-category1-B
fold 0


../Functions/prepro.py:187: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  indices = df1_.iloc[fold][df1_[col] == content].index-1  # fold内のある種別を持つインデックスの抽出
../Functions/prepro.py:188: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tgt_sum = df1_.iloc[out_fold][df1_[col] == content]["y"].sum()
../Functions/prepro.py:189: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tgt_size = df1_.iloc[out_fold][df1_[col] == content]["y"].shape[0]
/opt/anaconda3/envs/spla/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


fold 1
fold 2
fold 3
fold 4


../Functions/prepro.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[tgt_col][df2[col] == content] = df1[df1[col] == content][tgt_col].mean()


In [9]:
# make input

drop_cols = ["id", "lobby", "lobby-mode",  "period", "game-ver", "A1-weapon", "A2-weapon", "A3-weapon", "A4-weapon", \
              "B1-weapon", "B2-weapon", "B3-weapon", "B4-weapon"]
X, test_X = prepro.make_input(train_df, test_df, drop_cols, categorical_encode = False, verbose=False)

In [10]:
categorical_features_indices = np.where(X.dtypes == "object")[0]

In [11]:
# 全データを5つに分割
SIZE = X.shape[0]
K = 5

def make_kfolds(SIZE, K):
    # return list object, each element is indices of its fold
    FOLD_SIZE = int(SIZE/K)
    res = []
    indices = [i for i in range(SIZE)]
    for i in range(K-1):
        fold = random.sample(indices, FOLD_SIZE)
        indices = list(set(indices) - set(fold))
        res.append(fold)
    res.append(indices)
    return res

folds = make_kfolds(SIZE, K)
print(len(folds))
for i, fold in enumerate(folds):
    print("fold ", i+1, " size is ", len(fold))
    

    
if SIZE != len(set(sum(folds, []))):
    print("error is occuring in spliting")
else :
    print("successfully split")


5
fold  1  size is  26450
fold  2  size is  26450
fold  3  size is  26450
fold  4  size is  26450
fold  5  size is  26450
successfully split


In [12]:
params = {
    "loss_function" : "Logloss",
    "eval_metric" : "Logloss",
    "iterations":1000,
    "learning_rate" : 0.1,
    "use_best_model": True,
    "random_seed":1,

    

}

THRESHOLD = 0.50
models = []
cv_scores = []
temp = 0
train_pred = []

all_indices = sum(folds, [])
for i in range(K):
    valid_indices = folds[i]
    train_indices = list(set(all_indices) - set(valid_indices))
    # print("train ", len(train_indices), " , valid ", len(valid_indices))
    
    train_X = X.iloc[train_indices]
    train_y = y.iloc[train_indices]
    valid_X = X.iloc[valid_indices]
    valid_y = y.iloc[valid_indices]
    
    train_data = Pool(train_X, train_y, cat_features=categorical_features_indices)
    valid_data = Pool(valid_X, valid_y, cat_features=categorical_features_indices)
    
    model = CatBoostClassifier(**params)

    
    model.fit(
        train_data,
        eval_set=valid_data,
        early_stopping_rounds=10,
        verbose=10,
        use_best_model=True,
    )
    
    pred = model.predict(valid_X)
    # pred = np.where(pred < THRESHOLD, 0, 1)
    train_pred.append(pred)
    temp += np.sum(pred)
    
    score = accuracy_score(pred, valid_y)
    
    models.append(model)
    cv_scores.append(score)
    
    
print("cv score : ", np.mean(cv_scores))    
print("cv ratio : ", temp / SIZE)

0:	learn: 0.6923233	test: 0.6923194	best: 0.6923194 (0)	total: 611ms	remaining: 10m 9s
10:	learn: 0.6881699	test: 0.6887875	best: 0.6887875 (10)	total: 4.35s	remaining: 6m 31s
20:	learn: 0.6858904	test: 0.6871894	best: 0.6871894 (20)	total: 9.9s	remaining: 7m 41s
30:	learn: 0.6840615	test: 0.6860122	best: 0.6860122 (30)	total: 17.9s	remaining: 9m 21s
40:	learn: 0.6825643	test: 0.6851113	best: 0.6851113 (40)	total: 25.5s	remaining: 9m 55s
50:	learn: 0.6810388	test: 0.6843002	best: 0.6843002 (50)	total: 33.5s	remaining: 10m 24s
60:	learn: 0.6800714	test: 0.6838627	best: 0.6838627 (60)	total: 42.4s	remaining: 10m 52s
70:	learn: 0.6790300	test: 0.6835097	best: 0.6835097 (70)	total: 48.6s	remaining: 10m 35s
80:	learn: 0.6781791	test: 0.6831766	best: 0.6831766 (80)	total: 53.3s	remaining: 10m 4s
90:	learn: 0.6772487	test: 0.6829538	best: 0.6829538 (90)	total: 57.7s	remaining: 9m 36s
100:	learn: 0.6762926	test: 0.6827459	best: 0.6827459 (100)	total: 1m 1s	remaining: 9m 9s
110:	learn: 0.675248

0:	learn: 0.6921696	test: 0.6922282	best: 0.6922282 (0)	total: 420ms	remaining: 6m 59s
10:	learn: 0.6883310	test: 0.6891032	best: 0.6891032 (10)	total: 4.99s	remaining: 7m 28s
20:	learn: 0.6859451	test: 0.6873623	best: 0.6873623 (20)	total: 10.4s	remaining: 8m 3s
30:	learn: 0.6844124	test: 0.6862709	best: 0.6862709 (30)	total: 15.6s	remaining: 8m 7s
40:	learn: 0.6827894	test: 0.6852943	best: 0.6852943 (40)	total: 20.5s	remaining: 7m 59s
50:	learn: 0.6815485	test: 0.6846645	best: 0.6846645 (50)	total: 25.6s	remaining: 7m 56s
60:	learn: 0.6804014	test: 0.6841404	best: 0.6841404 (60)	total: 31.2s	remaining: 8m
70:	learn: 0.6795286	test: 0.6838416	best: 0.6838416 (70)	total: 37s	remaining: 8m 3s
80:	learn: 0.6787664	test: 0.6834974	best: 0.6834974 (80)	total: 42.5s	remaining: 8m 2s
90:	learn: 0.6778913	test: 0.6832002	best: 0.6832002 (90)	total: 47.4s	remaining: 7m 53s
100:	learn: 0.6770324	test: 0.6828928	best: 0.6828928 (100)	total: 52.9s	remaining: 7m 51s
110:	learn: 0.6761580	test: 0.6

In [13]:
preds = []
for i in range(K):
    model = models[i]
    pred = model.predict(test_X)
    preds.append(pred)
    print(np.sum(pred) / pred.shape[0])
    
preds = np.array(preds)
preds = np.mean(preds, axis=0)
print(np.sum(preds) / preds.shape[0])


temp = pd.DataFrame({"pred":pred})
# temp.hist(bins=20)

0.5861326746647848
0.5515525758645025
0.5624911785462244
0.5700776287932251
0.6088567395906845
0.5758221594918842


In [14]:
preds = np.where(preds < THRESHOLD, 0, 1)
print(np.sum(preds) / preds.shape[0])

submit_df = pd.DataFrame({'y': preds})
submit_df.index.name = 'id'
submit_df.to_csv('../Submissions/submission_cat_2_{}.csv'.format(K))

0.5791813690896259


# モデル解釈

In [20]:
importance = pd.DataFrame(models[0].get_feature_importance(), index=X.columns, columns=['importance']).sort_values('importance', ascending=False)
display(importance)

,importance
A-level-median,3.705585
A-level-mean,2.692150
B-level-median,2.669680
stage-tgt-enc,1.996567
B-level-sum,1.894585
B-level-mean,1.686152
A1-level,1.669624
team-category1-A-tgt-enc,1.666641
B-level-max,1.493040
team-special-A,1.422769


In [ ]:
train_df["pred"] = 0
train_df["y"] = y.values
for i in range(K):
    train_df["pred"].iloc[folds[i]] = train_pred[i]

In [ ]:
result_df = train_df[train_df["y"] != train_df["pred"]]

In [ ]:
print(result_df[result_df["mode"] == "nawabari"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "hoko"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "asari"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "area"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "yagura"].shape[0]/result_df.shape[0])

In [ ]:
result_df[result_df["mode"] == "nawabari"]